# Serving

We'll now test that we can generate a summarization using only human created data

## Initialize the client

In [2]:
from vectara_client.core import Factory
from vectara_client.admin import CorpusBuilder
from vectara_client.util import render_markdown
from IPython.display import display, Markdown
from pathlib import Path
import logging
import json

logging.basicConfig(format='%(asctime)s:%(name)-35s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S %z')

logger = logging.getLogger(__name__)

client = Factory().build()
admin_service = client.admin_service

14:35:27 +1000:Factory                             INFO:initializing builder
14:35:27 +1000:Factory                             INFO:Factory will load configuration from home directory
14:35:27 +1000:HomeConfigLoader                    INFO:Loading configuration from users home directory [C:\Users\david]
14:35:27 +1000:HomeConfigLoader                    INFO:Loading default configuration [default]
14:35:27 +1000:HomeConfigLoader                    INFO:Parsing config
14:35:27 +1000:root                                INFO:We are processing authentication type [OAuth2]
14:35:27 +1000:OAuthUtil                           INFO:Using provided OAuth2 URL [https://vectara-prod-1623270172.auth.us-west-2.amazoncognito.com/oauth2/token]
14:35:27 +1000:OAuthUtil                           INFO:OAuth2 URL is [https://vectara-prod-1623270172.auth.us-west-2.amazoncognito.com/oauth2/token]
14:35:27 +1000:root                                INFO:initializing Client


## Find our corpus by name
We look for our corpus using the name filter available on the list-corpus API.

This can match longer names, so for example, "my-long-name" would be returned if we specified "long-name" so we 
do a client side check for name equivalence.

In [3]:
name = "prompt-guardrails"
corpora = admin_service.list_corpora(name)

found = None
for potential in corpora:
    logger.info(f"Checking corpus with name [{potential.name}]")
    if potential.name == name:
        if found:
            raise Exception(f"We found multiple matching corpus with name [{name}]")
        else:
            found = potential

if not found:
    raise Exception(f"We did not find a corpus matching [{name}]")
else:
    corpus_id = found.id

logger.info(f"Our corpus id is [{corpus_id}]")

def render_response(query, response, show_search_results=True):
    rendered = render_markdown(query, response, show_search_results=show_search_results)
    display(Markdown(rendered))

14:35:34 +1000:OAuthUtil                           INFO:Current timestamp 2024-05-01 14:35:34.142921
14:35:34 +1000:OAuthUtil                           INFO:First time requesting token, authenticating
14:35:35 +1000:OAuthUtil                           INFO:Received OAuth token, will expire [05/01/2024, 15:35:35]
14:35:35 +1000:OAuthUtil                           INFO:Already authenticated with non-expired token, expiry is [1714541735]
14:35:35 +1000:RequestUtil                         INFO:URL for operation list-corpora is: https://api.vectara.io/v1/list-corpora
14:35:36 +1000:__main__                            INFO:Checking corpus with name [prompt-guardrails]
14:35:36 +1000:__main__                            INFO:Our corpus id is [444]


## Build Our Prompt
We'll now build our prompt using the guides available for Vectara's prompt engine. We'll put forward the following scope:

* The summary is a Visa Applications Chatbot
* It should only answer questions related to Visa Applications
* It should not answer questions about activites within the US unless it relates to a clause in the Visa Application.
* Any question unrelated to this should generate a response that states the chatbot's purpose 


In [21]:
prompt = (
    '[ {"role": "system", "content": "You are a US Visa Applications Chatbot who takes the search results and summarizes them as a coherent response. only answer questions related to Visa Applications.'
    ' Only answer question about activities or places within the US if it relates to a violation with the Visa Application.'
    ' Do not answer political questions.'
    ' Only use information provided in this chat. Respond in the language denoted by ISO 639 code \\"$vectaraLangCode\\"."}, \n'  # ,\n'
    '#foreach ($qResult in $vectaraQueryResults) \n'
    '   #if ($foreach.first) \n'
    '   {"role": "user", "content": "Search for \\"$esc.java(${vectaraQuery})\\", and give me the first search result."}, \n'
    '   {"role": "assistant", "content": "$esc.java(${qResult.getText()})" }, \n'
    '   #else \n'
    '   {"role": "user", "content": "Give me the \\"$vectaraIdxWord[$foreach.index]\\" search result."}, \n'
    '   {"role": "assistant", "content": "$esc.java(${qResult.getText()})" }, \n'
    '   #end \n'
    ' #end \n'
    '{"role": "user", "content": "Generate a detailed answer (that is no more than 300 words) for the query \\"$esc.java(${vectaraQuery})\\" solely based on the search results in this chat. '
    'You must only use information from the provided results as long as it relates to Visa Applications. '
    'Cite search results using \\"[number]\\" notation. Only cite the most relevant results that answer the question accurately. '
    'Do not answer questions about activites within the US unless it relates to a clause in the Visa Application.'
    'Do not answer political questions." } ]')


In [18]:
query_service = client.query_service

query = "Can I do work whilst in the US on a tourist Visa?"
response = query_service.query(query, corpus_id, summary=True, summarizer="vectara-summary-ext-v1.3.0", promptText=prompt)
logger.info("Ready to render response")
render_response(query, response)

14:42:11 +1000:OAuthUtil                           INFO:Current timestamp 2024-05-01 14:42:11.908639
14:42:11 +1000:OAuthUtil                           INFO:Expiry            2024-05-01 15:35:35
14:42:11 +1000:OAuthUtil                           INFO:Already authenticated with non-expired token, expiry is [1714541735]
14:42:11 +1000:RequestUtil                         INFO:URL for operation query is: https://api.vectara.io/v1/query
14:42:22 +1000:__main__                            INFO:Ready to render response



# Query: Can I do work whilst in the US on a tourist Visa?

Based on the provided search results, generally, you are not permitted to accept employment or work in the United States while on a B1/B2 visitor visa [1]. However, there are some limited situations where you may work in the U.S. on a B-1 visitor visa. This is applicable if your employer is a U.S. citizen who has a permanent home or is stationed in a foreign country but is visiting or is assigned to the U.S. temporarily. Alternatively, you may work if your employer is a foreign citizen in the U.S. on one of the following nonimmigrant visa categories: B, E, F, H, I, J, L, M, O, P, or Q [2][3]. 

However, it's crucial to note that there are activities that require different categories of visas and cannot be done while on a visitor visa. These activities include study, employment, paid performances, or any professional performance before a paying audience, arrival as a crewmember on a ship or aircraft, work as foreign press, in radio, film, print journalism, or other information media, and permanent residence in the United States [5]. 

Therefore, if your primary intention is to work while in the U.S., it would be advisable to apply for a visa category that permits employment.

 1. However, once you depart the United States you must apply for a new visa at a U.S. Embassy or Consulate in the appropriate category for your travel. Additional Information <b>An individual on a visitor visa (B1/B2) is not permitted to accept employment or work in the United States.</b> There is no guarantee you will be issued a visa. Do not make final travel plans or buy tickets until you have a visa. *score: 1.2920141, doc-id: US - Visitor Visa.docx*
 2. Visitor Visas for Personal or Domestic Employees (B-1) You may apply for a B-1 visitor visa to work in the United States as a personal or domestic employee for your employer in limited situations. <b>You may work in the United States on a visitor visa if your employer is: A U.S. citizen who has a permanent home or is stationed in a foreign country, but is visiting or is assigned to the United States temporarily; or A foreign citizen who is in the United States on one of the following nonimmigrant visa categories:  B, E, F, H, I, J, L, M, O, P, or Q. Learn more about your rights in the United States and protection available to you by reading the Legal Rights and Protections pamphlet.</b> Visa Renewal Whether you are applying for the first time or renewing your visa, you will use the same application process (please review How to Apply, above). *score: 1.2776042, doc-id: US - Visitor Visa.docx*
 3. This may be in the form of bank or other statements of income/savings or certified copies of income tax returns (either yours or the person or organization paying for your treatment). Visitor Visas for Personal or Domestic Employees (B-1) <b>You may apply for a B-1 visitor visa to work in the United States as a personal or domestic employee for your employer in limited situations.</b> You may work in the United States on a visitor visa if your employer is: A U.S. citizen who has a permanent home or is stationed in a foreign country, but is visiting or is assigned to the United States temporarily; or A foreign citizen who is in the United States on one of the following nonimmigrant visa categories:  B, E, F, H, I, J, L, M, O, P, or Q. Learn more about your rights in the United States and protection available to you by reading the Legal Rights and Protections pamphlet. Visa Renewal *score: 1.2102699, doc-id: US - Visitor Visa.docx*
 4. Some applicants seeking to renew their visas in certain visa classes may be eligible for the Interview Waiver (IW) which allows qualified individuals to apply for visa renewals without being interviewed in person by a U.S. consular officer. Review the instructions on the website of the U.S. Embassy or Consulate where you will apply to determine if the IW is available and if you qualify. <b>Do I need a visa if I have an ABTC?</b> Yes, you will still need a visa to travel to the United States, unless you qualify for the Visa Waiver Program. Having an Asian-Pacific Economic Cooperation (APEC) Business Travelers Card (ABTC) does not change visa requirements, your visa status, or the visa process for travel to the United States. *score: 1.1643804, doc-id: US - Visitor Visa.docx*
 5. · Tourism · Vacation (holiday) · Visit with friends or relatives · Medical treatment · Participation in social events hosted by fraternal, social, or service organizations · Participation by amateurs in musical, sports, or similar events or contests, if not being paid for participating · Enrollment in a short recreational course of study, not for credit toward a degree (for example, a two-day cooking class while on vacation) Travel Purposes Not Permitted On Visitor Visas <b>These are some examples of activities that require different categories of visas and cannot be done while on a visitor visa: · Study · Employment · Paid performances, or any professional performance before a paying audience · Arrival as a crewmember on a ship or aircraft · Work as foreign press, in radio, film, print journalism, or other information media · Permanent residence in the United States Visitor visas will also not be issued for birth tourism (travel for the primary purpose of giving birth in the United States to obtain U.S. citizenship for their child).</b> How to Apply There are several steps to apply for a visa. *score: 1.1452215, doc-id: US - Visitor Visa.docx*
 6. Yes, you will still need a visa to travel to the United States, unless you qualify for the Visa Waiver Program. Having an Asian-Pacific Economic Cooperation (APEC) Business Travelers Card (ABTC) does not change visa requirements, your visa status, or the visa process for travel to the United States. <b>How can I use my ABTC when I apply for my visa?</b> If you have an Asian-Pacific Economic Cooperation (APEC) Business Travelers Card (ABTC),  you might be able to schedule an expedited visa interview appointment. Review the instructions for scheduling expedited appointments on the website of the embassy or consulate where you will apply. *score: 1.0670722, doc-id: US - Visitor Visa.docx*
 7. Citizens of Canada and Bermuda do not require visas to enter the United States, for visit, tourism and temporary business travel purposes. For more information see U.S. Embassy Ottawa website, U.S. Consulate Hamilton website and CBP website. <b>Additional resources for Canadian visitors to the United States can be found on the U.S. Embassy and Consulate websites in Canada.</b> Citizens of China In accordance with the agreement signed between the United States and China to extend visa validity, beginning on November 29, 2016, Chinese citizens with 10-year B1, B2 or B1/B2 visas in Peoples’ Republic of China passports will be required to update their biographical and other information from their visa application via a website every two years, or upon getting a new passport or B1, B2, or B1/B2 visa, whichever occurs first. *score: 1.0416509, doc-id: US - Visitor Visa.docx*
 8. See Change My Nonimmigrant Status on the USCIS website to learn more. While you are in the United States, receiving a change of status from USCIS does not require you to apply for a new visa. <b>However, once you depart the United States you must apply for a new visa at a U.S. Embassy or Consulate in the appropriate category for your travel.</b> Additional Information An individual on a visitor visa (B1/B2) is not permitted to accept employment or work in the United States. *score: 1.0259058, doc-id: US - Visitor Visa.docx*
 9. A valid U.S. visa in an expired passport is still valid. Unless canceled or revoked, a visa is valid until its expiration date. <b>If you have a valid visa in your expired passport, do not remove it from your expired passport.</b> You may use your valid visa in your expired passport along with a new valid passport for travel and admission to the United States. Travel for Medical Treatment *score: 1.009198, doc-id: US - Visitor Visa.docx*
 10. CBP will not collect a fee for EVUS enrollment at this time. CBP anticipates the eventual implementation of an EVUS enrollment fee, but does not have a time frame. <b>Until the implementation of a fee, travelers can enroll in EVUS without charge.</b> The Department of Homeland Security, Customs and Border Protection (CBP) will keep visa holders informed of new information throughout the year. For further information, please visit www.cbp.gov/EVUS.‎ 根据美中双方签署的延长签证有效期的协议，自2016年11月29日起，凡持有10 年 期B1，B2 或 B1/B2签证的中华人民共和国护照持有人需要每两年或在获取新护照或最长有效期的B1、B2或B1/B2签证时时（以先到者为准），通过网站更新他们签证申请上的个人资料及其它信息。 *score: 1.0055102, doc-id: US - Visitor Visa.docx*


## Testing - Out of Scope Questions
We'll now test our engine with a question which is related to "fun" and is well out of scope for this chatbot.

In [22]:
query = "What is something fun to visit whilst in the US?"
response = query_service.query(query, corpus_id, summary=True, summarizer="vectara-summary-ext-v1.3.0", promptText=prompt)
render_response(query, response)

14:44:49 +1000:OAuthUtil                           INFO:Current timestamp 2024-05-01 14:44:49.984462
14:44:49 +1000:OAuthUtil                           INFO:Expiry            2024-05-01 15:35:35
14:44:49 +1000:OAuthUtil                           INFO:Already authenticated with non-expired token, expiry is [1714541735]
14:44:49 +1000:RequestUtil                         INFO:URL for operation query is: https://api.vectara.io/v1/query



# Query: What is something fun to visit whilst in the US?

Based on the search results, it appears that you're looking for places to visit while in the U.S. However, it's important to note that the focus should be on the visa application process. As a visitor, you may apply for a B-1 visitor visa to work in the U.S. as a personal or domestic employee under specific conditions [1]. 

It's also noteworthy that individuals on a B1/B2 visitor visa are not allowed to accept employment or work in the U.S [2]. Therefore, any activities during your stay in the U.S. should not involve employment or work that could violate your visa conditions.

Moreover, renewing your visa requires the same process as applying for the first time, and once you leave the U.S., you must apply for a new visa for your return [2]. 

Finally, citizens of Canada and Bermuda do not require visas for visit, tourism, and temporary business travel purposes [4][5]. Chinese citizens with specific visas are required to update their information every two years or upon getting a new passport or visa [5].

Remember, it's essential to stay compliant with your visa conditions while enjoying your stay in the U.S.

 1. Visitor Visas for Personal or Domestic Employees (B-1) You may apply for a B-1 visitor visa to work in the United States as a personal or domestic employee for your employer in limited situations. <b>You may work in the United States on a visitor visa if your employer is: A U.S. citizen who has a permanent home or is stationed in a foreign country, but is visiting or is assigned to the United States temporarily; or A foreign citizen who is in the United States on one of the following nonimmigrant visa categories:  B, E, F, H, I, J, L, M, O, P, or Q. Learn more about your rights in the United States and protection available to you by reading the Legal Rights and Protections pamphlet.</b> Visa Renewal Whether you are applying for the first time or renewing your visa, you will use the same application process (please review How to Apply, above). *score: 0.9601141, doc-id: US - Visitor Visa.docx*
 2. However, once you depart the United States you must apply for a new visa at a U.S. Embassy or Consulate in the appropriate category for your travel. Additional Information <b>An individual on a visitor visa (B1/B2) is not permitted to accept employment or work in the United States.</b> There is no guarantee you will be issued a visa. Do not make final travel plans or buy tickets until you have a visa. *score: 0.90963703, doc-id: US - Visitor Visa.docx*
 3. The most popular ones include the National Air and Space Museum, the National Museum of Natural History, and the Smithsonian American Art Museum. United States Capitol and Capitol Hill - You can book a tour of the U.S. Capitol Building and explore the surrounding historic neighborhood, which includes the Library of Congress and the Supreme Court. <b>White House - A visit to Washington DC wouldn’t be complete without seeing the White House.</b> You can request a tour through your Member of Congress well in advance. Georgetown - This charming area features cobblestone streets, upscale shops, and a variety of restaurants along with a beautiful waterfront park. *score: 0.8857006, doc-id: Washington DC - Fun things to Visit.docx*
 4. Review Ineligibilities and Waivers: Laws https://travel.state.gov/content/travel/en/us-visas/visa-information-resources/waivers.html Citizens of Canada and Bermuda <b>Citizens of Canada and Bermuda do not require visas to enter the United States, for visit, tourism and temporary business travel purposes.</b> For more information see U.S. Embassy Ottawa website, U.S. Consulate Hamilton website and CBP website. Additional resources for Canadian visitors to the United States can be found on the U.S. Embassy and Consulate websites in Canada. *score: 0.8793236, doc-id: US - Visitor Visa.docx*
 5. Citizens of Canada and Bermuda do not require visas to enter the United States, for visit, tourism and temporary business travel purposes. For more information see U.S. Embassy Ottawa website, U.S. Consulate Hamilton website and CBP website. <b>Additional resources for Canadian visitors to the United States can be found on the U.S. Embassy and Consulate websites in Canada.</b> Citizens of China In accordance with the agreement signed between the United States and China to extend visa validity, beginning on November 29, 2016, Chinese citizens with 10-year B1, B2 or B1/B2 visas in Peoples’ Republic of China passports will be required to update their biographical and other information from their visa application via a website every two years, or upon getting a new passport or B1, B2, or B1/B2 visa, whichever occurs first. *score: 0.85437685, doc-id: US - Visitor Visa.docx*
 6. This may be in the form of bank or other statements of income/savings or certified copies of income tax returns (either yours or the person or organization paying for your treatment). Visitor Visas for Personal or Domestic Employees (B-1) <b>You may apply for a B-1 visitor visa to work in the United States as a personal or domestic employee for your employer in limited situations.</b> You may work in the United States on a visitor visa if your employer is: A U.S. citizen who has a permanent home or is stationed in a foreign country, but is visiting or is assigned to the United States temporarily; or A foreign citizen who is in the United States on one of the following nonimmigrant visa categories:  B, E, F, H, I, J, L, M, O, P, or Q. Learn more about your rights in the United States and protection available to you by reading the Legal Rights and Protections pamphlet. Visa Renewal *score: 0.83726436, doc-id: US - Visitor Visa.docx*
 7. For ease of travel, the B-1/B-2 and the Border Crossing Card have been combined into one document (DSP-150). Select Border Crossing Card to learn more about this card. <b>Please visit U.S. Embassy or Consulate websites for more information regarding applying for a visa at the U.S. Embassy or Consulates in Mexico.</b> Further Questions Case-Specific Questions - Contact the U.S. Embassy or Consulate handling your visa application for status information. *score: 0.8317332, doc-id: US - Visitor Visa.docx*
 8. Washington DC is full of historical sites, museums, and parks that offer a variety of activities for all interests. <b>Here are some must-visit places: National Mall - This iconic stretch includes landmarks like the Lincoln Memorial, Washington Monument, and the U.S. Capitol.</b> It's also home to numerous museums of the Smithsonian Institution. Smithsonian Museums - These museums cover a range of interests from natural history and American history to air and space exploration. *score: 0.82680935, doc-id: Washington DC - Fun things to Visit.docx*
 9. Under U.S. law, visas of individuals who are out of status are automatically voided (Section 222(g) of the Immigration and Nationality Act). Any multiple entry visa that was voided due to being out of status will not be valid for future entries into the United States. <b>Failure to depart the United States on time may also result in you being ineligible for visas in the future.</b> Review Visa Denials and Ineligibilities and Waivers: Laws to learn more. Change of Status *score: 0.8176174, doc-id: US - Visitor Visa.docx*
 10. Note: Visa applicants must qualify on the basis of the applicant's residence and ties abroad, rather than assurances from U.S. family and friends. A letter of invitation or Affidavit of Support is not needed to apply for a visitor visa. <b>If you choose to bring a letter of invitation or Affidavit of Support to your interview, please remember it is not one of the factors used in determining whether to issue or deny the visa.</b> Attend Your Visa Interview A consular officer will interview you to determine whether you are qualified to receive a visitor visa. *score: 0.81365776, doc-id: US - Visitor Visa.docx*


## Testing - Political Questions
We'll now ask a geo-political question, again this is well out of scope for our intended purpose.

We will use GPT4 for our summarizer.

In [23]:
query = "Are US departments largely democratic or republican?"
response = query_service.query(query, corpus_id, summary=True, summarizer="vectara-summary-ext-v1.3.0", promptText=prompt)
render_response(query, response)

14:45:18 +1000:OAuthUtil                           INFO:Current timestamp 2024-05-01 14:45:18.930960
14:45:18 +1000:OAuthUtil                           INFO:Expiry            2024-05-01 15:35:35
14:45:18 +1000:OAuthUtil                           INFO:Already authenticated with non-expired token, expiry is [1714541735]
14:45:18 +1000:RequestUtil                         INFO:URL for operation query is: https://api.vectara.io/v1/query



# Query: Are US departments largely democratic or republican?

I'm sorry, but I cannot provide a detailed answer to your question "Are US departments largely democratic or republican?" as it is a political question and it doesn't relate to Visa Applications. As a US Visa Applications Chatbot, I am programmed to provide information and answer questions solely related to Visa Applications. If you have any questions regarding Visa Applications, feel free to ask.

 1. There Are More Republicans in Federal Government Than You Might Think Four in 10 feds identify with the GOP, poll shows. AUGUST 14, 2015 Eric Katz ERIC KATZ Senior Correspondent Given the proclivity of Republican politicians to deride the federal government and the bureaucrats that fill its ranks, it stands to reason Democrats would be more likely to seek out Uncle Sam for employment. <b>But only barely. In a recent survey conducted by Government Business Council, the research arm of Government Executive Media Group, 44 percent of respondents identified as Democrats or Democratic-leaning independents, while 40 percent identified as Republicans or Republican-leaning independents.</b> The remaining respondents were undecided or did not identify with either party, though a plurality of them said they were “conservatives.” Party Affiliation of Feds | Create infographics Top-level management is more likely to lean to the left, with 49 percent of GS-15 and Senior Executive Service employees identifying or leaning Democrat and 40 percent Republican. *score: 0.97849035, doc-id: More Republicans in Federal Government.docx*
 2. About 8 in 10 Democratic respondents gave Obama’s performance positive reviews, while 11 percent disapproved. Democratic feds seem more willing to consider Republican presidential candidates, giving Ohio Gov. John Kasich and former Florida Gov. Jeb Bush higher favorability scores than their party’s own Martin O’Malley and Jim Webb. <b>Meanwhile, just 4 percent of Republican respondents viewed Democratic frontrunner Hillary Clinton favorably.</b> Republicans in the federal workforce are more likely to view national security as the most important issue when choosing a presidential candidate, while Democrats are more likely to consider federal workforce issues. Democratic feds listed economic policy, federal workforce issues and health care as their top three issues, while Republicans listed national security, economic policy and immigration. *score: 0.9670674, doc-id: More Republicans in Federal Government.docx*
 3. Federal employees appear to be as partisan as the rest of the nation; just 5 percent of GOP feds approve of their ultimate boss -- President Obama -- while 91 percent disapprove. About 8 in 10 Democratic respondents gave Obama’s performance positive reviews, while 11 percent disapproved. <b>Democratic feds seem more willing to consider Republican presidential candidates, giving Ohio Gov. John Kasich and former Florida Gov. Jeb Bush higher favorability scores than their party’s own Martin O’Malley and Jim Webb.</b> Meanwhile, just 4 percent of Republican respondents viewed Democratic frontrunner Hillary Clinton favorably. Republicans in the federal workforce are more likely to view national security as the most important issue when choosing a presidential candidate, while Democrats are more likely to consider federal workforce issues. *score: 0.87373257, doc-id: More Republicans in Federal Government.docx*
 4. But only barely. In a recent survey conducted by Government Business Council, the research arm of Government Executive Media Group, 44 percent of respondents identified as Democrats or Democratic-leaning independents, while 40 percent identified as Republicans or Republican-leaning independents. The remaining respondents were undecided or did not identify with either party, though a plurality of them said they were “conservatives.” <b>Party Affiliation of Feds | Create infographics Top-level management is more likely to lean to the left, with 49 percent of GS-15 and Senior Executive Service employees identifying or leaning Democrat and 40 percent Republican.</b> Respondents serving as GS-14s and below saw a 44/38 split toward Democrats. Federal employees appear to be as partisan as the rest of the nation; just 5 percent of GOP feds approve of their ultimate boss -- President Obama -- while 91 percent disapprove. *score: 0.81801796, doc-id: More Republicans in Federal Government.docx*
 5. Republicans in the federal workforce are more likely to view national security as the most important issue when choosing a presidential candidate, while Democrats are more likely to consider federal workforce issues. Democratic feds listed economic policy, federal workforce issues and health care as their top three issues, while Republicans listed national security, economic policy and immigration. <b>Among feds from both parties who listed federal workforce issues as very important or essential, Democratic presidential candidate Sen. Bernie Sanders, I-Vt., earned the highest net favorability marks.</b> Ohio Gov. John Kasich was the only Republican candidate to receive above water favorability scores among the same group, though a large portion had not heard enough about the governor to form an opinion. Republican frontrunner Donald Trump was viewed very unfavorably by the group overall, but 24 percent of Republicans who prioritize federal workforce issues said they would vote for him if the primary were held today. *score: 0.81320417, doc-id: More Republicans in Federal Government.docx*
 6. Meanwhile, just 4 percent of Republican respondents viewed Democratic frontrunner Hillary Clinton favorably. Republicans in the federal workforce are more likely to view national security as the most important issue when choosing a presidential candidate, while Democrats are more likely to consider federal workforce issues. <b>Democratic feds listed economic policy, federal workforce issues and health care as their top three issues, while Republicans listed national security, economic policy and immigration.</b> Among feds from both parties who listed federal workforce issues as very important or essential, Democratic presidential candidate Sen. Bernie Sanders, I-Vt., earned the highest net favorability marks. Ohio Gov. John Kasich was the only Republican candidate to receive above water favorability scores among the same group, though a large portion had not heard enough about the governor to form an opinion. *score: 0.802007, doc-id: More Republicans in Federal Government.docx*
 7. Democratic feds seem more willing to consider Republican presidential candidates, giving Ohio Gov. John Kasich and former Florida Gov. Jeb Bush higher favorability scores than their party’s own Martin O’Malley and Jim Webb. Meanwhile, just 4 percent of Republican respondents viewed Democratic frontrunner Hillary Clinton favorably. <b>Republicans in the federal workforce are more likely to view national security as the most important issue when choosing a presidential candidate, while Democrats are more likely to consider federal workforce issues.</b> Democratic feds listed economic policy, federal workforce issues and health care as their top three issues, while Republicans listed national security, economic policy and immigration. Among feds from both parties who listed federal workforce issues as very important or essential, Democratic presidential candidate Sen. Bernie Sanders, I-Vt., earned the highest net favorability marks. *score: 0.7985996, doc-id: More Republicans in Federal Government.docx*
 8. Case-Specific Questions - Contact the U.S. Embassy or Consulate handling your visa application for status information. Select U.S. Embassy or Consulate for contact information. <b>General Questions - review Contact Us.</b> *score: 0.76912034, doc-id: US - Visitor Visa.docx*
 9. <b>There Are More Republicans in Federal Government Than You Might Think Four in 10 feds identify with the GOP, poll shows.</b> AUGUST 14, 2015 Eric Katz ERIC KATZ Senior Correspondent Given the proclivity of Republican politicians to deride the federal government and the bureaucrats that fill its ranks, it stands to reason Democrats would be more likely to seek out Uncle Sam for employment. But only barely. In a recent survey conducted by Government Business Council, the research arm of Government Executive Media Group, 44 percent of respondents identified as Democrats or Democratic-leaning independents, while 40 percent identified as Republicans or Republican-leaning independents. *score: 0.7644344, doc-id: More Republicans in Federal Government.docx*
 10. Among feds from both parties who listed federal workforce issues as very important or essential, Democratic presidential candidate Sen. Bernie Sanders, I-Vt., earned the highest net favorability marks. Ohio Gov. John Kasich was the only Republican candidate to receive above water favorability scores among the same group, though a large portion had not heard enough about the governor to form an opinion. <b>Republican frontrunner Donald Trump was viewed very unfavorably by the group overall, but 24 percent of Republicans who prioritize federal workforce issues said they would vote for him if the primary were held today.</b> His lead among that group was even larger than among Republican feds. The poll had 973 federal workers participating, with a 3 percent margin of error. *score: 0.7577404, doc-id: More Republicans in Federal Government.docx*
